In [1]:
using Gurobi, JuMP, DataFrames, CSV
using Statistics
using MLBase
using JuMP, Ipopt
using StatsBase
using Juniper
using Shuffle

In [104]:
#import the DataFramea
df = CSV.read("Cleaning/clustering_data.csv", DataFrame)

# Only keep rows where sold ==1
df = df[df[!,"sold"] .== 1, :];

#Keep the rows where price is between 5th and 95th percentile
df = df[df[!,"sales_price"] .> quantile(df[!,"sales_price"], 0.05), :]
df = df[df[!,"sales_price"] .< quantile(df[!,"sales_price"], 0.95), :];

#Drop the column sold
df = df[:, Not(:sold)]

#Define target variable 
y = df[!,"sales_price"]
X= df[:, Not(:sales_price)];

X

,dated,height,inscribed,signed,stamped,width,aspect_ratio,area,low_estimate
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,40.94,1.0,1.0,0.0,15.55,2.63,636.62,320750.0
2,0.0,41.73,1.0,1.0,0.0,25.39,1.64,1059.52,513200.0
3,0.0,47.24,1.0,1.0,0.0,22.36,2.11,1056.29,513200.0
4,0.0,50.79,0.0,1.0,0.0,27.76,1.83,1409.93,1.9245e6
5,0.0,52.76,1.0,1.0,0.0,27.56,1.91,1454.07,320750.0
6,0.0,23.82,0.0,1.0,0.0,47.24,0.5,1125.26,44910.0
7,0.0,35.43,0.0,1.0,0.0,37.87,0.94,1341.73,1.283e6
8,0.0,20.87,0.0,0.0,0.0,23.03,0.91,480.64,76980.0
9,0.0,19.09,1.0,1.0,0.0,22.24,0.86,424.56,384900.0


In [270]:
seed = 15095
(X_train, y_train), (X_test, y_test) =  IAI.split_data(:classification, X, y, seed=seed, train_proportion=0.8); #further splitting train set into train and validation sets

In [105]:
function corrPair(threshold)
    n,p = size(X_train)
    pairs = []
    for i=1:p
        for j=i+1:p
            if abs(cor(X_train[:,i], X_train[:,j])) > threshold
                append!(pairs, [(i,j)])
            end
        end
    end
    return collect(pairs)
end

HC_pairs = corrPair(0.7)
HC_pairs

9-element Vector{Any}:
 (2, 8)
 (6, 8)
 (9, 10)
 (25, 79)
 (29, 82)
 (44, 97)
 (51, 92)
 (100, 103)
 (107, 108)

In [106]:
#Which are these variables?
for i in corrPair(0.7)
    println(names(X_train)[i[1]], " ", names(X_train)[i[2]])
end

height area
width area
low_estimate high_estimate
Egon Schiele Austria
Fernando Botero Colombia
Le Pho Vietnam
Pablo Ruiz Picasso Spain
Christies Sothebys
painting paper


In [273]:
function compute_mse(X, y, ß)
    n,p = size(X)
    return sum((Matrix(X)*ß .- y).^2)/n
end;

In [326]:
function optimize(X_training, y_training)
    n, p = size(X_training)
    #£ = [0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0, 250.0]

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 60) #Set a timelimit of 60 seconds

    @variable(model, ß[i=1:p])
    @variable(model, z[i=1:p], Bin)
    @variable(model, a[i=1:p]>=0)
    
    M=100000000
    k=10

    #@constraint(model, binary[i=1:p], -M*z[i]≤ß[i])
    #@constraint(model, binary2[i=1:p], ß[i]≤M*z[i])
    #@constraint(model, corr[(i,j) in HC_pairs], z[i] + z[j] ≤ 1)
    #@constraint(model, lim, sum(z) ≤ 15)
    #@constraint(model, absoluteValue, å .≥ ß)
    #@constraint(model, absoluteValue1, å .≥ -ß)

    #Constraints for sparsity - Big M
    @constraint(model, [j=1:p], ß[j] <= M*z[j])
    @constraint(model, [j=1:p], -M*z[j] <= ß[j])
    
    #Constraint for sparsity 2 - Limiting number of non zero components
    @constraint(model, sum(z[j] for j=1:p) <= k)

    #Constraint for features with high collinearities
    @constraint(model, [j=1:length(HC_pairs)], (z[HC_pairs[j][1]] + z[HC_pairs[j][2]]) <= 1)

    #Insert constraints - Linearize norm 1
    @constraint(model,[j=1:p], ß[j]<=a[j])
    @constraint(model,[j=1:p], -ß[j]<=a[j])
     
    lambda = 7
    best = Inf
    
    #= for £ in lambdas
        @objective(model, Min, 1/n * sum((y_training - (Matrix(X_training) * ß)).^2) + £ * sum(å))
        optimize!(model)
        score = compute_mse(Matrix(X_valid), y_valid, JuMP.value.(ß))
        println("Lambda: ", £, " MSE: ", score)
        if score < best
            best = score 
            lambda = £
        end
    end  =#
 
    @objective(model, Min, 1/n * sum(((y_training) - (Matrix(X_training) * ß)).^2) + lambda * sum(a))
    
    optimize!(model)

    return JuMP.value.(ß), lambda
end

optimize (generic function with 1 method)

In [322]:
betas, best_lambda=optimize(X_train, y_train)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 600
Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 459 rows, 336 columns and 1028 nonzeros
Model fingerprint: 0xb13a45e7
Model has 2051 quadratic objective terms
Variable types: 224 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [7e+00, 1e+12]
  QObjective range [1e-04, 2e+12]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 7.414203e+11
Presolve time: 0.00s
Presolved: 459 rows, 336 columns, 1028 nonzeros
Presolved model has 2051 quadratic objective terms
Variable types: 224 continuous, 112 in

([12822.064931663697, -57.540881669265616, 12711.78993127409, 27551.601070997505, 0.0, 611.7551010045381, 19496.675722712636, 0.0, 1.1388068887513585, 0.04255559195873042  …  36152.335244052636, 0.0, 0.0, 0.0, 38055.1907327329, 0.0, 0.0, 0.0, 0.0, 0.0], 7)

In [323]:
#Calculate R2 score
function r2_score(y_true, y_pred)
    SS_res = sum((y_true .- y_pred).^2)
    SS_tot = sum((y_true .- mean(y_true)).^2)
    return 1 - SS_res/SS_tot
end

r2_score(y_test, Matrix(X_test) * betas)

0.7914833193124997

In [324]:
#Which is the non zero coefficient?
for i in 1:length(betas)
    if betas[i] != 0
        println(names(X_train)[i])
    end
end

dated
height
inscribed
signed
width
aspect_ratio
low_estimate
high_estimate
Ivan Konstantinovich Aivazovsky
René Magritte
artist_third_tier
China
Sothebys
painting


In [325]:
sum(betas.!=0)

14

# Looks like Estimate Low alone can achieve an R2 of 0.76, it is just multiplying the low estimate by 1.3423. Let's try it all over again but without these estimate variables. 

In [155]:
#import the DataFramea
df = CSV.read("Cleaning/clustering_data.csv", DataFrame)

# Only keep rows where sold ==1
df = df[df[!,"sold"] .== 1, :];

#Drop the "low_estimate" and "high_estimate" columns
df = df[:, Not(:low_estimate)]
df = df[:, Not(:high_estimate)]
df = df[:, Not(:sold)]

#Keep the rows where price is between 5th and 95th percentile
df = df[df[!,"sales_price"] .> quantile(df[!,"sales_price"], 0.05), :]
df = df[df[!,"sales_price"] .< quantile(df[!,"sales_price"], 0.95), :];

#Define target variable 
y = df[!,"sales_price"]
X= df[:, Not(:sales_price)];

seed = 15095
(X2, y2), (X_test, y_test) = IAI.split_data(:classification, X, y, seed=seed, train_proportion=0.8) #setting apart test set
(X_train, y_train), (X_valid, y_valid) =  IAI.split_data(:classification, X2, y2, seed=seed, train_proportion=0.8); #further splitting train set into train and validation sets

function corrPair(threshold)
    n,p = size(X_train)
    pairs = []
    for i=1:p
        for j=i+1:p
            if abs(cor(X_train[:,i], X_train[:,j])) > threshold
                append!(pairs, [(i,j)])
            end
        end
    end
    return collect(pairs)
end

HC_pairs = corrPair(0.7)

8-element Vector{Any}:
 (2, 8)
 (6, 8)
 (23, 77)
 (27, 80)
 (56, 78)
 (74, 92)
 (98, 101)
 (105, 106)

In [172]:
function optimize2(X_training, y_training)
    n, p = size(X_training)

    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "TimeLimit", 120) #Set a timelimit of 60 seconds

    @variable(model, ß[i=1:p])
    @variable(model, z[i=1:p], binary=true)
    @variable(model, å[i=1:p]>=0)
    
    M=100000000

    @constraint(model, binary[i=1:p], -M*z[i]≤ß[i])
    @constraint(model, binary2[i=1:p], ß[i]≤M*z[i])
    @constraint(model, corr[(i,j) in HC_pairs], z[i] + z[j] ≤ 1)
    @constraint(model, lim, sum(z) ≤ 25)
    @constraint(model, absoluteValue, å .≥ ß)
    @constraint(model, absoluteValue1, å .≥ -ß)

    lambda = 7
    best = 100000000000
 
    @objective(model, Min, sum(((y_training) - (Matrix(X_training) * ß)).^2) + lambda * sum(å))
    
    optimize!(model)

    return JuMP.value.(ß), lambda
end

optimize2 (generic function with 1 method)

In [173]:
betas, best_lambda=optimize2(X_train, y_train)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-17
Set parameter TimeLimit to value 120
Set parameter TimeLimit to value 120
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 449 rows, 330 columns and 1006 nonzeros
Model fingerprint: 0xd07c9143
Model has 1822 quadratic objective terms
Variable types: 220 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [7e+00, 6e+13]
  QObjective range [4e+00, 5e+11]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 2.224347e+16
Presolve time: 0.00s
Presolved: 449 rows, 330 columns, 1006 nonzeros
Presolved model has 1822 quadratic objective terms
Variable types: 220 continuous, 110 in

([0.0, 0.0, 0.0, 0.0, 0.0, 3164.742892905001, 0.0, 0.0, 432359.16655160487, -2.9103830456733704e-11  …  0.0, 0.0, 0.0, 0.0, 279978.26585591177, 0.0, 0.0, 0.0, 200016.98681021814, 0.0], 7)

In [174]:
r2_score(y_test, Matrix(X_test) * betas)

0.07312473435333489

In [171]:
r2_score(y_train, Matrix(X_train) * betas)

0.07824213248317413

In [170]:
#Which is the non zero coefficient?
for i in 1:length(betas)
    if betas[i] != 0
        println(names(X_train)[i])
    end
end

height
aspect_ratio
Alfred Sisley
Andy Warhol
Bernard Buffet
Chu Teh Chun (zhu Dequn)
Chu Teh-Chun
Claude Monet
Emil Nolde
Fernand Léger
Gerhard Richter
Henry Moore
Jean Michel Basquiat
Lucio Fontana
Maqbool Fida Husain
Max Ernst
Paul Klee
René Magritte
Serge Poliakoff
Wu Guanzhong
Zao Wou-Ki
Zhang Daqian
Austria
China
England
France
Germany
Indonesia
Italy
Netherlands
Russia
Spain
Switzerland
USA
United States of America
Vietnam
other_country
Christies
Villa Grisebach Auktionen
paper
photograph


# Holistic Logistic Regression

In [212]:
seed = 15095

#import the DataFramea
df = CSV.read("Cleaning/clustering_data.csv", DataFrame);
#get column names
show(names(df))


["dated", "height", "inscribed", "signed", "stamped", "width", "aspect_ratio", "area", "sold", "sales_price", "low_estimate", "high_estimate", "Alberto Giacometti", "Alexej Von Jawlensky", "Alfred Sisley", "Alighiero Boetti", "Andy Warhol", "Bernard Buffet", "Camille Jacob Pissarro", "Chu Teh Chun (zhu Dequn)", "Chu Teh-Chun", "Cindy Sherman", "Claude Monet", "Cy Twombly", "Damien Hirst", "Edgar Degas", "Egon Schiele", "Emil Nolde", "Eugène Louis Boudin", "Fernand Léger", "Fernando Botero", "Francis Newton Souza", "Francis Picabia", "Gerhard Richter", "Giorgio De Chirico", "Henri Emile Benoit Matisse", "Henry Moore", "Ivan Konstantinovich Aivazovsky", "Jean Dubuffet", "Jean Michel Basquiat", "Joan Miró", "Josef Albers", "Kees Van Dongen", "Keith Haring", "Laurence Stephen Lowry", "Le Pho", "Lucio Fontana", "Maqbool Fida Husain", "Marc Chagall", "Maurice De Vlaminck", "Maurice Utrillo", "Max Ernst", "Pablo Ruiz Picasso", "Paul Klee", "Pierre Auguste Renoir", "Pierre Bonnard", "Pu Ru", "

In [214]:
features_lr= ["dated", "height", "inscribed", "signed" , "stamped", "width", "aspect_ratio", "area", "sold", "sales_price", "low_estimate", "high_estimate"]

12-element Vector{String}:
 "dated"
 "height"
 "inscribed"
 "signed"
 "stamped"
 "width"
 "aspect_ratio"
 "area"
 "sold"
 "sales_price"
 "low_estimate"
 "high_estimate"

In [215]:
df= df[:, features_lr]

,dated,height,inscribed,signed,stamped,width,aspect_ratio,area,sold
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,0.0,33.66,1.0,1.0,0.0,16.93,1.99,569.86,1
2,0.0,13.58,1.0,1.0,0.0,10.43,1.3,141.64,0
3,0.0,40.94,1.0,1.0,0.0,15.55,2.63,636.62,1
4,0.0,13.19,1.0,1.0,0.0,16.73,0.79,220.67,0
5,0.0,9.45,1.0,1.0,0.0,10.63,0.89,100.45,1
6,0.0,55.91,1.0,1.0,0.0,28.54,1.96,1595.67,1
7,0.0,41.73,1.0,1.0,0.0,25.39,1.64,1059.52,1
8,0.0,47.24,1.0,1.0,0.0,22.36,2.11,1056.29,1
9,0.0,50.79,0.0,1.0,0.0,27.76,1.83,1409.93,1


In [216]:
df_sold = df[df[!,"sold"] .== 1, :]
df_unsold = df[df[!,"sold"] .== 0, :]

#Change the 0's for -1's in the "sold" column
df_unsold[!,"sold"] = df_unsold[!,"sold"] .- 1

df_sold_limited = df_sold[sample(1:nrow(df_sold), 862, replace=false), :] #select only 862 sold rows to get a balanced dataset

df_lr = vcat(df_unsold, df_sold_limited) #concatenate the two dataframes

#Distribution of sold column - Check it worked
countmap(df_lr[!,"sold"])

Dict{Int64, Int64} with 2 entries:
  -1 => 862
  1  => 862

In [217]:
#Define target variable 
y = df_lr[!,"sold"]
X= df_lr[:, Not(:sales_price)];
X= X[:, Not(:sold)];

seed = 15095

(X_train, y_train), (X_test, y_test) =  IAI.split_data(:classification, X, y, seed=seed, train_proportion=0.8); 

In [218]:
#Distribution in y_train
countmap(y_train)

Dict{Int64, Int64} with 2 entries:
  -1 => 690
  1  => 690

In [219]:
countmap(y_test)

Dict{Int64, Int64} with 2 entries:
  -1 => 172
  1  => 172

In [240]:
function optimize3(X_training, y_training)

    n, p = size(X_training)

    #nl_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0)
    #minlp_solver = optimizer_with_attributes(Juniper.Optimizer, "nl_solver"=>nl_solver)
    #model = Model(minlp_solver)
    model = Model(Ipopt.Optimizer)

    @variable(model, ß[i=1:p])
    @variable(model, ß_0)
    #@variable(model, z[i=1:p], Bin)
    
    M=1000
    k=25
    lambda = 1.0

    #Constraints for sparsity - Big M
    #@NLconstraint(model, [j=1:p], ß[j] <= M*z[j])
    #@NLconstraint(model, [j=1:p], -M*z[j] <= ß[j])
    
    #Constraint for sparsity 2 - Limiting number of non zero components
    #@NLconstraint(model, sum(z[j] for j=1:p) <= k)

    #Constraint for features with high collinearities
    #@constraint(model, [j=1:length(HC_pairs)], (z[HC_pairs[j][1]] + z[HC_pairs[j][2]]) <= 1)

    #Insert constraints - Linearize norm 1
    #@NLconstraint(model,[j=1:p], ß[j]<=a[j])
    #@NLconstraint(model,[j=1:p], -ß[j]<=a[j])
     
    #@NLobjective(model, Min, sum(log(1 + exp(-y_training[i]*(sum(X[i,j]*ß[j] for j=1:p)+ß_0))) for i=1:n))
    @NLobjective(model, Min, sum(log(1 + exp(-y_training[i]*(sum(X[i,j]*ß[j] for j=1:p)+ß_0)+ lambda * sum(abs(ß[j]) for j=1:p))) for i=1:n))
    
    optimize!(model)

    return JuMP.value.(ß), JuMP.value.(ß_0), lambda
end

optimize3 (generic function with 1 method)

In [241]:
betas, beta_0, best_lambda=optimize3(X_train, y_train)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       66

Total number of variables............................:       11
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.5654311e+02 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

([1.0825166704382923e-17, -1.1749545012354683e-16, -1.5382229219656644e-16, 1.1340916441768522e-16, 3.7366526801669746e-16, -1.520782407796195e-18, -5.612865373309016e-17, 2.7995155537271794e-18, -1.1869131307202603e-20, 1.140524358295424e-20], -1.0791423579083306e-15, 1.0)

In [242]:
#Predictions - Binary Classification from logistic regression
y_pred = 1 ./ (1 .+ exp.(-(Matrix(X_test)*betas .+ beta_0)))
show(y_pred)

[0.49999999999999933, 0.5000000000000011, 0.49999999999999933, 0.4999999999999998, 0.5, 0.49999999999999933, 0.4999999999999991, 0.49999999999999956, 0.49999999999999933, 0.49999999999999956, 0.49999999999999956, 0.49999999999999956, 0.499999999999996, 0.4999999999999989, 0.4999999999999988, 0.49999999999999956, 0.4999999999999998, 0.4999999999999998, 0.49999999999999956, 0.4999999999999998, 0.5000000000000006, 0.49999999999999933, 0.49999999999999933, 0.49999999999999956, 0.4999999999999992, 0.4999999999999999, 0.49999999999999967, 0.49999999999999933, 0.49999999999999956, 0.4999999999999998, 0.4999999999999998, 0.500000000000007, 0.49999999999999956, 0.4999999999999998, 0.5000000000000011, 0.49999999999999956, 0.49999999999999933, 0.5, 0.49999999999999933, 0.49999999999999956, 0.5000000000000018, 0.4999999999999991, 0.49999999999999933, 0.4999999999999992, 0.5000000000000006, 0.4999999999999989, 0.4999999999999998, 0.49999999999999867, 0.4999999999999992, 0.49999999999999933, 0.49999

In [243]:
#Convert to 0's and 1's
y_pred = ifelse.(y_pred .> 0.5, 1, -1)

#Accuracy
accuracy = sum(y_pred .== y_test) / length(y_test)

#Confusion Matrix
confusion_matrix = MLBase.countmap(zip(y_pred, y_test))


Dict{Tuple{Int64, Int64}, Int64} with 4 entries:
  (1, -1)  => 31
  (1, 1)   => 59
  (-1, -1) => 141
  (-1, 1)  => 113

In [244]:
accuracy

0.5813953488372093

# Optimal Prescriptive Tree

In [170]:
seed = 15095
#import the DataFramea
df = CSV.read("Cleaning/clustering_data.csv", DataFrame);

#Keep the rows where the low estimate is between 0 and 250000 -> 80% of the unsold artwork is in this range
df = df[df[!,"low_estimate"] .< 250000, :];

df_sold = df[df[!,"sold"] .== 1, :]
df_unsold = df[df[!,"sold"] .== 0, :];

df_sold_limited = df_sold[sample(1:nrow(df_sold), 251, replace=false), :] #select only some sold rows to get a balanced dataset 75/25

df_opt = vcat(df_unsold, df_sold_limited) #concatenate the two dataframes

#Distribution of sold column - Check it worked
countmap(df_opt[!,"sold"])

Dict{Int64, Int64} with 2 entries:
  0 => 699
  1 => 251

In [171]:
#write this dataframe to a csv file
#CSV.write("Cleaning/optimalPrescriptiveTree_251_data.csv", df_opt)

In [172]:
seed = 12345
X = df_opt[:, Not(:sold)];
#X = X[:, Not(:sales_price)];
X = X[:, Not(:high_estimate)];
X = X[:, Not(:low_estimate)];

y = df_opt.sold #outcome -> sold or not
t = df_opt.low_estimate; #treatments -> prices -> low estimate

In [173]:
(train_X, train_t, train_y), (test_X, test_t, test_y) = IAI.split_data(
    :policy_maximize, X, t, y, train_proportion=0.5, seed=seed);

In [174]:
train_t_discrete = round.(train_t ./ 5000, digits=0) .* 5000 #discretize the prices to the nearest 5k
train_t_discrete;

In [175]:
# Recover real revenues from sales_price
train_y_revenue= train_X.sales_price
test_y_revenue= test_X.sales_price

# Remove sales_price from the data
train_X = train_X[:, Not(:sales_price)]
test_X = test_X[:, Not(:sales_price)];

In [176]:
prescriptive_grid = IAI.GridSearch(
    IAI.OptimalTreePrescriptionMaximizer(
      random_seed=seed,
    ),
    max_depth=3:8,
)
#train_y_revenue = train_y .* train_t #revenue, which is prices times whether the product was purchased or not
IAI.fit!(prescriptive_grid, train_X, train_t_discrete, train_y_revenue)

All Grid Results:

 Row │ max_depth  cp         train_score  valid_score  rank_valid_score 
     │ Int64      Float64    Float64      Float64      Int64            
─────┼──────────────────────────────────────────────────────────────────
   1 │         3  0.0460251     0.667241   -7.45796e6                 6
   2 │         4  0.0474245     0.690664   -7.43289e6                 5
   3 │         5  0.0468913     0.707009   -7.22291e6                 1
   4 │         6  0.0330681     0.712274   -7.38708e6                 3
   5 │         7  0.0479886     0.715602   -7.26362e6                 2
   6 │         8  0.054017      0.721813   -7.40767e6                 4

Best Params:
  cp => 0.046891282926548344
  max_depth => 5

Best Model - Fitted OptimalTreePrescriptionMaximizer:
  1) Split: Andy Warhol < 0.5
    2) Split: Spain < 0.5
      3) Split: artist_third_tier < 0.5
        4) Split: Fernando Botero < 0.5
          5) Prescribe: 200000.0, 386 points, error 0.0006685
          6) Prescribe: 200000.0, 1 points, error 0.001977
        7) Split: Italy < 0.5
          8) Prescribe: 105000.0, 71 points, error 0.0005026
          9) Prescribe: 200000.0, 4 points, error 0.0004352
      10) Prescribe: 240000.0, 9 points, error 0.00171
    11) Prescribe: 155000.0, 4 points, error 0.0009669

In [177]:
t_options = collect(0:5000:250000) #range from min to max in 500 k increments
t_options;

In [178]:
reward_lnr = IAI.NumericClassificationRewardEstimator(
    outcome_estimator=IAI.XGBoostClassifier(num_round=10),
    outcome_insample_num_folds=2,
    reward_estimator=:direct_method,
    estimation_kernel_bandwidth=1,
    random_seed=seed,
)

function get_rewards(reward_lnr, X, t, y, t_options)
  predictions, score = IAI.fit_predict!(reward_lnr, X, t, y, t_options,
                                        outcome_score_criterion=:auc)
  rewards = predictions[:reward]
  for t in t_options
    rewards[!, Symbol(t)] = round.(rewards[!, Symbol(t)] .* t, digits=3)
  end
  rewards, score
end
train_rewards, train_reward_score = get_rewards(reward_lnr, train_X, train_t,
                                                train_y, t_options)
train_rewards;

In [179]:
test_rewards, test_reward_score = get_rewards(reward_lnr, test_X, test_t,
                                              test_y, t_options)
test_rewards;

In [180]:
function evaluate(recommendations, outcomes, actual_revenue)
    n = length(recommendations)
    pred_revenue = [outcomes[i, recommendations[i]] for i in 1:n]
  
    improvement = mean(pred_revenue .- actual_revenue) / mean(actual_revenue)
  end
  
  # Get revenue observed for test set in reality
  test_revenue = test_y .* test_t;
  #test_revenue = df_opt.sales_price

In [181]:
recommendations_pres = Symbol.(round.(Int, IAI.predict(prescriptive_grid, test_X)[1]))
evaluate(recommendations_pres, test_rewards, test_revenue)

1.2479409647478101